In [27]:
#%pip install wikitextparser

import xml.etree.ElementTree as ET
import wikitextparser as wtp
from docx import Document
from docx.enum.text import WD_ALIGN_PARAGRAPH
from tqdm import tqdm
import pandas as pd
from heapq import heappush,heappop
import os
from concurrent.futures import ThreadPoolExecutor

In [2]:
tree = ET.parse('ruwiki-20241201-pages-articles-multistream1.xml')
root = tree.getroot()
len(root)

In [3]:
prefix = '{http://www.mediawiki.org/xml/export-0.11/}'

In [ ]:
# Выходная папка
output_dir = "output"

if not os.path.exists(output_dir):
	os.makedirs(output_dir)
	

In [ ]:
# Преобработка

indexies = []

pages = root.findall(f'{prefix}page')
'''
for i in range(len(pages)):
	text = pages[i].find(f"{prefix}revision").find(f"{prefix}text").text
	if(text and text.find("{|")):
		indexies.append(i)
'''

xml.etree.ElementTree.Element

In [52]:

def parse_to_docx(parsed : wtp.WikiText, title : str, filename : str) -> None:
	elements = []

	sections,tables,lists = parsed.get_sections(),parsed.get_tables(),parsed.get_lists()

	for i in range(len(sections)):
		heappush(elements,(sections[i].span[0],"section",i))

	for i in range(len(tables)):
		heappush(elements,(tables[i].span[0],"table",i))

	for i in range(len(lists)):
		heappush(elements,(lists[i].span[0],"list",i))
		
	doc = Document()
	doc.add_heading(f"Содержимое страницы: {title}", level = 1)

	while elements:
		elem = heappop(elements)

		if elem[1] == "section":
			sec = sections[elem[2]]

			if sec.title:
				secText = wtp.remove_markup(sec.title).strip()
				if secText:
					doc.add_heading(secText, level=sec.level)
		if elem[1] == "table":
			tbl = tables[elem[2]]

			doc.add_paragraph(f"Таблица: {tbl.caption}").alignment = WD_ALIGN_PARAGRAPH.CENTER

			data = tbl.data()

			if(len(data) <= 0):
				continue

			table = doc.add_table(rows=len(data), cols=len(data[0]))

			for i, row in enumerate(data):
				for j, cell in enumerate(row):
					if not cell:
						continue

					cellText = wtp.remove_markup(cell).strip()

					if cellText:
						table.cell(i, j).text = cellText
		elif elem[1] == "list":
			lst = lists[elem[2]]

			for item in lst.items:
				itemText = wtp.remove_markup(item).strip()

				if itemText:
					doc.add_paragraph(itemText, style="List Bullet")

	doc.save(os.path.join(output_dir, filename))

def parse_to_markdown(parsed : wtp.WikiText, title : str, filename : str) -> None:
	elements = []

	sections,tables,lists = parsed.get_sections(),parsed.get_tables(),parsed.get_lists()

	for i in range(len(sections)):
		heappush(elements,(sections[i].span[0],"section",i))

	for i in range(len(tables)):
		heappush(elements,(tables[i].span[0],"table",i))

	for i in range(len(lists)):
		heappush(elements,(lists[i].span[0],"list",i))
		
	doc = Document()
	doc.add_heading(f"Содержимое страницы: {title}", level = 1)

	with open(os.path.join(output_dir, filename),"w", encoding="utf-8") as mdf:
		mdf.write(f"# Содержимое страницы: {title}")
		
		while elements:
			elem = heappop(elements)

			if elem[1] == "section":
				sec = sections[elem[2]]

				if sec.title:
					secText = wtp.remove_markup(sec.title).strip()
					if secText:
						mdf.write(f"{'#' * sec.level} {secText}\n")
			if elem[1] == "table":
				tbl = tables[elem[2]]

				data = tbl.data()

				if(len(data) <= 0):
					continue

				df = pd.DataFrame(data[1:],columns=data[0])

				for i, row in enumerate(data):
					for j, cell in enumerate(row):
						if not cell:
							continue

						cellText = wtp.remove_markup(cell).strip()

						if cellText:
							if i == 0:
								df.columns.values[j] = cellText
							else:
								df.at[i-1,df.columns[j]] = cellText

				mdf.write(df.to_markdown(index=False))
				mdf.write('\n')
			elif elem[1] == "list":
				lst = lists[elem[2]]

				for item in lst.items:
					itemText = wtp.remove_markup(item).strip()

					if itemText:
						mdf.write(f"- {itemText}\n")

		mdf.write("\n---\n")

In [58]:
def parse_page(page : ET.Element) -> None:
	title = page.find(f"{prefix}title").text

	filename = "".join(x for x in title if x.isalnum()) + ".md"

	#if os.path.isfile(os.path.join(output_dir, filename)):
	#	return

	text = page.find(f"{prefix}revision").find(f"{prefix}text").text #.split("\n")

	if not text:
		return

	parsed = wtp.parse(text)

	parse_to_markdown(parsed,title,filename)

In [59]:
for page in tqdm(pages[0:], desc='Обработка всех страниц', total=len(pages)):
	parse_page(page)

Обработка всех страниц:   0%|          | 401/130477 [00:29<2:40:23, 13.52it/s]


KeyboardInterrupt: 

In [ ]:
# Парсим этот пиздец
with ThreadPoolExecutor(max_workers=10) as executor:
	list(tqdm(executor.map(parse_page, pages), desc='Создание страниц:', total=len(pages)))